# Rock Paper Scissors Lizard Spock

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
February 2025  

------

## Introduction

It is easy to make a simple [Rock-Paper-Scissors](https://en.wikipedia.org/wiki/Rock_paper_scissors) game graph using the Raku package ["Graph"](https://raku.land/zef:antononcube/Graph), [AAp1]. Here is such a graph in which the arrow directions indicate which item (vertex) wins:

In [20]:
#%html
Graph.new(<🪨 ✂️ ✂️ 📃 📃 🪨>.Hash.pairs):d
==> {.dot(engine => 'neato', :3graph-size):svg}()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- ✂️ -->
 
 ✂️ 
 
 ✂️ 
 
<!-- 📃 -->
 
 📃 
 
 📃 
 
<!-- ✂️->📃 -->
 
 ✂️->📃 
 
 
 
<!-- 🪨 -->
 
 🪨 
 
 🪨 
 
<!-- 📃->🪨 -->
 
 📃->🪨 
 
 
 
<!-- 🪨->✂️ -->
 
 🪨->✂️

Easy, but now we want to:
- Add labels to the edges 
    - Like "🪨 crushes ✂️"
- Change the items/weapons
    - Snake-Centipede-Snail
- Make graphs for game _upgrades_
    - Rock-Paper-Scissors-Lizard-Spock
    - Rock-Paper-Scissors-Fire-Water
- Utilize Large Language Models (LLMs) 
    - In order to simplify the graph making

In this document (notebook) we show how to do all of the above points.

- The package "Graph" (via [Graphviz DOT](https://graphviz.org/doc/info/lang.html)) can produce SVG plots that are readily rendered in different environments.
- LLMs "know" the Rock-Paper-Scissors game and its upgrade. 
- LLMs know how to (mostly, reliably) translate to emojis.

------

## Setup

This notebook is a Raku-chatbook, hence, its Jupyter session pre-loads certain packages and LLM-personas.

In [1]:
# Preloaded in any chatbook
# use LLM::Functions;
# use LLM::Prompts;

# Preloaded in a user init file
# use Graph;

# For this concrete session
use Text::Emoji;

LLM configurations:

In [2]:
my $conf4o = llm-configuration('chat-gpt', model => 'gpt-4o', :4096max-tokens, temperature => 0.4);
my $conf4o-mini = llm-configuration('chat-gpt', model => 'gpt-4o-mini', :4096max-tokens, temperature => 0.4);

($conf4o, $conf4o-mini)».Hash».elems

(24 24)

### JavaScript D3 plotting setup

In [3]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [4]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

### Additional

In [120]:
sub game-table(Graph:D $g) {
    cross-tabulate($g.edges(:dataset), <from>, <to>)
    ==> -> %h { %h.map({ $_.key => ($g.vertex-list Z=> $_.value{$g.vertex-list}).Hash }).Hash }()
    ==> to-dataset(missing-value => '-')
    ==> -> %h { for $g.vertex-list { %h{$_}{$_} = ''}; %h }()
    ==> -> %h { $g.vertex-list.map({ [|%h{$_}, "" => $_].Hash }) }()
    ==> to-html(field-names => ["", |$g.vertex-list])
    ==> { .Str.subst('1', '+', :g).subst('(Any)', '-', :g) }()
}

&game-table

-------

## LLM request

In [5]:
# % chat raku
# Make an array the edges of a graph for the game Rock-Paper-Scissors-Lizard-Spock.
# Each edges is represented with a hash with the keys "from", "to", "label".
# The label corresponds to the action taken with the edge, like, "Paper covers Rock", "Paper disproves Spock".

()

----

## Plain text graph

Here we create the Rock-Paper-Scissors-Lizard-Spock graph generated with the LLM-magic cell above:

In [6]:
my @edges =
    { from => 'Rock',     to => 'Scissors',  label => 'Rock crushes Scissors' },
    { from => 'Scissors', to => 'Paper',     label => 'Scissors cuts Paper' },
    { from => 'Paper',    to => 'Rock',      label => 'Paper covers Rock' },
    { from => 'Rock',     to => 'Lizard',    label => 'Rock crushes Lizard' },
    { from => 'Lizard',   to => 'Spock',     label => 'Lizard poisons Spock' },
    { from => 'Spock',    to => 'Scissors',  label => 'Spock smashes Scissors' },
    { from => 'Scissors', to => 'Lizard',    label => 'Scissors decapitates Lizard' },
    { from => 'Lizard',   to => 'Paper',     label => 'Lizard eats Paper' },
    { from => 'Paper',    to => 'Spock',     label => 'Paper disproves Spock' },
    { from => 'Spock',    to => 'Rock',      label => 'Spock vaporizes Rock' }
;

my $g = Graph.new(@edges, :directed);

Graph(vertexes => 5, edges => 10, directed => True)

Here we make the edge labels:

In [7]:
my %edge-labels;
@edges.map({ %edge-labels{$_<from>}{$_<to>} = $_<label>.words[1] });

deduce-type(%edge-labels)

Assoc(Atom((Str)), Assoc(Atom((Str)), Atom((Str)), 2), 5)

Here we plot the graph:

In [8]:
#% html
$g.dot(
    :6graph-size, 
    :1edge-width, 
    arrow-size => 0.2,
    :4edge-font-size,
    :%edge-labels,
    node-width => 0.25, node-height => 0.15, 
    node-shape => 'ellipse', 
    node-color => 'DimGray',
    node-stroke-width => 0.4,
    :node-labels, 
    :5node-font-size,
    engine => 'neato',
    :svg
)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Lizard -->
 
 Lizard 
 
 Lizard 
 
<!-- Paper -->
 
 Paper 
 
 Paper 
 
<!-- Lizard->Paper -->
 
 Lizard->Paper 
 
 
 eats 
 
<!-- Spock -->
 
 Spock 
 
 Spock 
 
<!-- Lizard->Spock -->
 
 Lizard->Spock 
 
 
 poisons 
 
<!-- Rock -->
 
 Rock 
 
 Rock 
 
<!-- Paper->Rock -->
 
 Paper->Rock 
 
 
 covers 
 
<!-- Paper->Spock -->
 
 Paper->Spock 
 
 
 disproves 
 
<!-- Rock->Lizard -->
 
 Rock->Lizard 
 
 
 crushes 
 
<!-- Scissors -->
 
 Scissors 
 
 Scissors 
 
<!-- Rock->Scissors -->
 
 Rock->Scissors 
 
 
 crushes 
 
<!-- Scissors->Lizard -->
 
 Scissors->Lizard 
 
 
 decapitates 
 
<!-- Scissors->Paper -->
 
 Scissors->Paper 
 
 
 cuts 
 
<!-- Spock->Rock -->
 
 Spock->Rock 
 
 
 vaporizes 
 
<!-- Spock->Scissors -->
 
 Spock->Scissors 
 
 
 smashes

**Remark:** Currently the class `Graph` does not "deal" with edge labels, but some of its methods (like, `dot`) do.

------

## Emoji translations

We can translate the vertices of the graph above in several ways:

- Manually
- Using `to-emoji` of ["Text::Emoji"](https://raku.land/zef:lizmat/Text::Emoji), [EMp1]
- Via LLMs

In [9]:
my %additional = spock => to-emoji(':vulcan-salute:'), paper => to-emoji(":page-with-curl:");
say (:%additional);
@edges.map(*<from>).map({ $_ => to-emoji(":$_:", %additional) })

additional => {paper => 📃, spock => 🖖}


(Rock => 🪨 Scissors => ✂️ Paper => 📃 Rock => 🪨 Lizard => 🦎 Spock => 🖖 Scissors => ✂️ Lizard => 🦎 Paper => 📃 Spock => 🖖)

In [10]:
# my $res = llm-synthesize( llm-prompt("Emojify")($g.vertex-list), e => $conf4o-mini  );

()

In [11]:
# $res.split(/\s+/, :skip-empty)».trim.Hash

()

In [12]:
#`[
my $res = llm-synthesize( [
    llm-prompt("Emojify")($g.vertex-list), 
    'Make a JSON dictionary of the original words as keys and the emojis as values', 
    llm-prompt('NothingElse')('JSON') 
    ], 
    e => $conf4o-mini, 
    form => sub-parser('JSON'):drop 
);
]

()

------

## Emoji graph

Let us remake game graph using suitable emojis. Here are the corresponding egdes:

In [13]:
my @edges-emo =
    { from => '🪨', to => '✂️',   label => 'crushes' },
    { from => '✂️',  to => '📄',  label => 'cuts' },
    { from => '📄', to => '🪨',  label => 'covers' },
    { from => '🪨', to => '🦎',  label => 'crushes' },
    { from => '🦎', to => '🖖',  label => 'poisons' },
    { from => '🖖', to => '✂️',   label => 'smashes' },
    { from => '✂️',  to => '🦎',  label => 'decapitates' },
    { from => '🦎', to => '📄',  label => 'eats' },
    { from => '📄', to => '🖖',  label => 'disproves' },
    { from => '🖖', to => '🪨',  label => 'vaporizes' }
;

my $g-emo = Graph.new(@edges-emo, :directed);

Graph(vertexes => 5, edges => 10, directed => True)

Here we make the edge labels:

In [14]:
my %edge-labels;
@edges-emo.map({ %edge-labels{$_<from>}{$_<to>} = $_<label> });

deduce-type(%edge-labels)

Assoc(Atom((Str)), Assoc(Atom((Str)), Atom((Str)), 2), 5)

Here we plot the graph (using a variety of setup options):

In [15]:
#% html
$g-emo.dot(
    :6graph-size, 
    :1edge-width,
    :3edge-font-size,
    edge-color => 'LightSlateGray',
    node-width => 0.2, node-height => 0.2, 
    node-shape => 'circle', 
    :node-labels, 
    :8node-font-size,
    node-fill-color => '#1F1F1F',
    node-color => 'LightSlateGray',
    node-stroke-width => 0.6,
    arrow-size => 0.25,
    :%edge-labels,
    engine => 'neato',
    :svg
)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- ✂️ -->
 
 ✂️ 
 
 ✂️ 
 
<!-- 📄 -->
 
 📄 
 
 📄 
 
<!-- ✂️->📄 -->
 
 ✂️->📄 
 
 
 cuts 
 
<!-- 🦎 -->
 
 🦎 
 
 🦎 
 
<!-- ✂️->🦎 -->
 
 ✂️->🦎 
 
 
 decapitates 
 
<!-- 🖖 -->
 
 🖖 
 
 🖖 
 
<!-- 📄->🖖 -->
 
 📄->🖖 
 
 
 disproves 
 
<!-- 🪨 -->
 
 🪨 
 
 🪨 
 
<!-- 📄->🪨 -->
 
 📄->🪨 
 
 
 covers 
 
<!-- 🖖->✂️ -->
 
 🖖->✂️ 
 
 
 smashes 
 
<!-- 🖖->🪨 -->
 
 🖖->🪨 
 
 
 vaporizes 
 
<!-- 🦎->📄 -->
 
 🦎->📄 
 
 
 eats 
 
<!-- 🦎->🖖 -->
 
 🦎->🖖 
 
 
 poisons 
 
<!-- 🪨->✂️ -->
 
 🪨->✂️ 
 
 
 crushes 
 
<!-- 🪨->🦎 -->
 
 🪨->🦎 
 
 
 crushes

-----

## Chuck Norris defeats them all!

Consider the [image](https://www.merchandisingplaza.us/40488/2/T-shirts-Chuck-Norris-Chuck-Norris-Rock-Paper-Scissors-Lizard-Spock-TShirt-l.jpg) (from [www.merchandisingplaza.us](https://www.merchandisingplaza.us/Chuck-Norris/Chuck-Norris-Rock-Paper-Scissors-Lizard-Spock-TShirt-40488)):

![](https://www.merchandisingplaza.us/40488/2/T-shirts-Chuck-Norris-Chuck-Norris-Rock-Paper-Scissors-Lizard-Spock-TShirt-l.jpg)

Let us try to remake it with a graph plot. At this point we simply add a "foot to connection" to all five vertices in the graph(s) above:

In [21]:
my $chuck = "🦶🏻";
my $g-chuck = $g.clone.edge-add( ($chuck X=> $g.vertex-list).Array, :directed);

Graph(vertexes => 6, edges => 15, directed => True)

But we also have to rename the vertices to be hand-gestures:

In [22]:
$g-chuck .= vertex-replace( { Scissors => '✌🏻', Rock => '✊🏻', Lizard => '🤏🏻', Spock => '🖖🏻', 'Paper' => '✋🏻' } )

Graph(vertexes => 6, edges => 15, directed => True)

In order to ensure that we get an "expected" graph plot, we take the vertex coordinates of a [wheel graph](https://en.wikipedia.org/wiki/Wheel_graph) or compute them by hand. Here we do the latter:

In [23]:
my @vs = <✊🏻 🖖🏻 🤏🏻 ✌🏻 ✋🏻>;
my %vertex-coordinates = @vs.kv.map( -> $i, $v { $v => [cos(π/2 + $i * 2 * π / 5), sin(π/2 + $i * 2 * π / 5)] });
%vertex-coordinates<🦶🏻> = (0, 0);
$g-chuck.vertex-coordinates = %vertex-coordinates;

{✊🏻 => [6.123233995736766e-17 1], ✋🏻 => [0.9510565162951536 0.3090169943749472], ✌🏻 => [0.5877852522924729 -0.8090169943749476], 🖖🏻 => [-0.9510565162951535 0.3090169943749475], 🤏🏻 => [-0.5877852522924732 -0.8090169943749473], 🦶🏻 => (0 0)}

Here we plot the graph:

In [24]:
#% html
$g-chuck.dot(
    background => '#5f5b4f',
    graph-label => 'Chuck Norris Defeats All'.uc,
    font-color => '#b8aa79',
    :6graph-size, 
    :2edge-width,
    :4edge-font-size,
    edge-color => 'AntiqueWhite',
    node-width => 0.56, node-height => 0.56, 
    node-shape => 'circle', 
    :node-labels, 
    :38node-font-size,
    node-fill-color => '#b8aa79',
    node-color => 'Gray',
    node-stroke-width => 0.6,
    arrow-size => 0.26,
    engine => 'neato',
    :svg
)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 CHUCK NORRIS DEFEATS ALL 
<!-- ✊🏻 -->
 
 ✊🏻 
 
 ✊🏻 
 
<!-- 🤏🏻 -->
 
 🤏🏻 
 
 🤏🏻 
 
<!-- ✊🏻->🤏🏻 -->
 
 ✊🏻->🤏🏻 
 
 
 
<!-- ✌🏻 -->
 
 ✌🏻 
 
 ✌🏻 
 
<!-- ✊🏻->✌🏻 -->
 
 ✊🏻->✌🏻 
 
 
 
<!-- 🦶🏻 -->
 
 🦶🏻 
 
 🦶🏻 
 
<!-- 🦶🏻->✊🏻 -->
 
 🦶🏻->✊🏻 
 
 
 
<!-- 🖖🏻 -->
 
 🖖🏻 
 
 🖖🏻 
 
<!-- 🦶🏻->🖖🏻 -->
 
 🦶🏻->🖖🏻 
 
 
 
<!-- ✋🏻 -->
 
 ✋🏻 
 
 ✋🏻 
 
<!-- 🦶🏻->✋🏻 -->
 
 🦶🏻->✋🏻 
 
 
 
<!-- 🦶🏻->🤏🏻 -->
 
 🦶🏻->🤏🏻 
 
 
 
<!-- 🦶🏻->✌🏻 -->
 
 🦶🏻->✌🏻 
 
 
 
<!-- 🖖🏻->✊🏻 -->
 
 🖖🏻->✊🏻 
 
 
 
<!-- 🖖🏻->✌🏻 -->
 
 🖖🏻->✌🏻 
 
 
 
<!-- ✋🏻->✊🏻 -->
 
 ✋🏻->✊🏻 
 
 
 
<!-- ✋🏻->🖖🏻 -->
 
 ✋🏻->🖖🏻 
 
 
 
<!-- 🤏🏻->🖖🏻 -->
 
 🤏🏻->🖖🏻 
 
 
 
<!-- 🤏🏻->✋🏻 -->
 
 🤏🏻->✋🏻 
 
 
 
<!-- ✌🏻->✋🏻 -->
 
 ✌🏻->✋🏻 
 
 
 
<!-- ✌🏻->🤏🏻 -->
 
 ✌🏻->🤏🏻

### Matching the colors

We can use "LLM vision" to get the colors of the original image:

In [40]:
my $url = 'https://www.merchandisingplaza.us/40488/2/T-shirts-Chuck-Norris-Chuck-Norris-Rock-Paper-Scissors-Lizard-Spock-TShirt-l.jpg';
llm-vision-synthesize('What are the dominant colors in this image? Give them in hex code.', $url)

The dominant colors in the image are:

- Olive Green: #5B5F4C
- Beige: #D1C49A
- White: #FFFFFF
- Black: #000000

These colors are approximate and may vary slightly.

### Graph generating with LLMs

Instead of specifying the graph edges by hand, we can use LLMs. The results are not that good, but YMMV.

In [48]:
my $res2 =
llm-vision-synthesize([
    'Give the edges the graph for this image of Rock-Paper-Scissors-Lizard-Spock-Chuck -- use relevant emojis.',
    'Give the edges as an array of dictionaries. Each dictionary with keys "from" and "to".',
    llm-prompt('NothingElse')('JSON')
    ], 
    $url,
    e => $conf4o,
    form => sub-parser('JSON'):drop
    )

[{from => ✂️, to => 📄} {from => 📄, to => 🪨} {from => 🪨, to => 🦎} {from => 🦎, to => 🖖} {from => 🖖, to => ✂️} {from => ✂️, to => 🦎} {from => 🦎, to => 📄} {from => 📄, to => 🖖} {from => 🖖, to => 🪨} {from => 🪨, to => ✂️} {from => 🤜, to => ✂️} {from => 🤜, to => 📄} {from => 🤜, to => 🪨} {from => 🤜, to => 🦎} {from => 🤜, to => 🖖}]

In [52]:
#% html
Graph.new($res2, :directed).dot(:5graph-size, engine => 'neato', arrow-size => 0.5):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- ✂️ -->
 
 ✂️ 
 
 ✂️ 
 
<!-- 📄 -->
 
 📄 
 
 📄 
 
<!-- ✂️->📄 -->
 
 ✂️->📄 
 
 
 
<!-- 🦎 -->
 
 🦎 
 
 🦎 
 
<!-- ✂️->🦎 -->
 
 ✂️->🦎 
 
 
 
<!-- 🖖 -->
 
 🖖 
 
 🖖 
 
<!-- 📄->🖖 -->
 
 📄->🖖 
 
 
 
<!-- 🪨 -->
 
 🪨 
 
 🪨 
 
<!-- 📄->🪨 -->
 
 📄->🪨 
 
 
 
<!-- 🖖->✂️ -->
 
 🖖->✂️ 
 
 
 
<!-- 🖖->🪨 -->
 
 🖖->🪨 
 
 
 
<!-- 🤜 -->
 
 🤜 
 
 🤜 
 
<!-- 🤜->✂️ -->
 
 🤜->✂️ 
 
 
 
<!-- 🤜->📄 -->
 
 🤜->📄 
 
 
 
<!-- 🤜->🖖 -->
 
 🤜->🖖 
 
 
 
<!-- 🤜->🦎 -->
 
 🤜->🦎 
 
 
 
<!-- 🤜->🪨 -->
 
 🤜->🪨 
 
 
 
<!-- 🦎->📄 -->
 
 🦎->📄 
 
 
 
<!-- 🦎->🖖 -->
 
 🦎->🖖 
 
 
 
<!-- 🪨->✂️ -->
 
 🪨->✂️ 
 
 
 
<!-- 🪨->🦎 -->
 
 🪨->🦎

-----

## Other game upgrades

Here is a table of upgraded game:

In [122]:
#% html
game-table($g-chuck)

,✊🏻,✋🏻,✌🏻,🖖🏻,🤏🏻,🦶🏻
✊🏻,,-,+,-,+,-
✋🏻,+,,-,+,-,-
✌🏻,-,+,,-,+,-
🖖🏻,+,-,+,,-,-
🤏🏻,-,+,-,+,,-
🦶🏻,+,+,+,+,+,


-----

## Image generation 

In [ ]:
#% dalle, size=landscape
Show a three panel Japanese painting in the style of Yamato-E of geishas playing the game "kitsune-ken".

In [ ]:
#% dalle, size=landscape
Show a three panel Japanese painting in the style of Wood-block print of geishas playing the game "kitsune-ken".
Make sure shown are only three geishas. Make sure the hands and hand gestures are correct.


-----

## References

### Packages

[AAp1] Anton Antonov, 
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov, 
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov, 
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[EMp1] Elizabeth Mattijsen,
[Text::Emoji Raku package](https://github.com/lizmat/Text-Emoji),
(2024-2025),
[GitHub/lizmat](https://github.com/lizmat).

### Videos

[AAv1] Anton Antonov,
["Upgrading Epidemiological Models into War Models"](https://www.youtube.com/watch?v=852vMS_6Qaw),
(2024),
[YouTube/@WolframResearch](https://www.youtube.com/@WolframResearch).